# Transfep Dataset: Analytics and Feature Selection

This notebook performs data analysis on the last partition of the dataset, since it is the longest. The first 2 partitions are reserved for validation/testing.

## Imports & Constants

In [ ]:
%matplotlib widget

%load_ext autoreload
%autoreload

import os

import polars as pl

from scipy import fft
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import GradientBoostingRegressor

import joblib
from pickle import HIGHEST_PROTOCOL

import transfep_utils

from matplotlib.axes import Axes
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_theme(
  "paper", "whitegrid",
  font="roboto", font_scale=2
)

# Given the behaviour of full_solar, it makes sense to model log(1+y)
# instead of the original signal
PRODUCTION_AS_LOG = True

DF = transfep_utils.get_dataset_n(2)

if PRODUCTION_AS_LOG:
  DF = DF.with_columns(
    pl.col("full_solar").log1p().alias("full_solar")
  )

TARGETS = ["energy", "full_solar"]
VARIABLES = [
    col for col in DF.columns
    if col != "datetime" and col not in TARGETS
]

RELEVANT_FREQS = [
    # 1., Hourly not possible due to sampling rate (Nyquist)
    1. / 12, # Bi-Daily
    1. / 24,  # Daily
    1. / (24 * 7),  # Weekly
    1. / (24 * 30), # Monthly
    1. / (24 * 365) # Yearly
]

## Visualization (Temporal/Spectral)

Separate plots for target variables and independent variables.

In [ ]:
def highlight_freq(ax: Axes, freq: float, tol: float = 1e-2, alpha=1e-1):
    ax.axvspan(
        xmin=freq - tol * freq,
        xmax=freq + tol * freq,
        alpha=alpha,
        color="red"
    )


### Spectral DataFrame

In [ ]:
df_spectral = pl.DataFrame(
    [
        pl.Series(
            "frequency",
            fft.rfftfreq(DF.height)
        )
    ] + [
        pl.Series(
            col + "_fft",
            np.abs(fft.rfft(DF.get_column(col).to_numpy())),
            dtype=pl.Float64
        )
        for col in VARIABLES + TARGETS
    ]
)

### Target Visualization

- Daily and Weekly periodicities (Bi-daily is just a harmonic of daily)
- Dataset is too short to observe yearly periodicity in the spectrum
  - This will be further studied in feature analysis
- Makes more sense to model the logarithm of full-solar
  - Periodicity preserved when transformed

In [ ]:
fig_1, axes_1 = plt.subplots(
    2, 1,
    sharex="col",
    figsize=(12, 6)
)
fig_1.tight_layout(pad=2)

fig_2, axes_2 = plt.subplots(
    2, 1,
    sharex="col",
    figsize=(12, 6)
)
fig_2.tight_layout(pad=2)

axes = [[ax1, ax2] for (ax1, ax2) in zip(axes_1, axes_2)]

for idx, var in enumerate(TARGETS):
    time_plot = sns.lineplot(
        DF,
        x="datetime",
        y=var,
        ax=axes[idx][0]
    )
    for tick_idx, label in enumerate(time_plot.get_xticklabels()):
        if tick_idx % 2 == 0:
            label.set_visible(True)
        else:
            label.set_visible(False)

    sns.lineplot(
        df_spectral,
        x="frequency",
        y=var + "_fft",
        ax=axes[idx][1]
    )
    axes[idx][1].set_xlabel("frequency [$h^{-1}$]")
    axes[idx][1].set_xscale("log")
    axes[idx][1].set_yscale("log")

    for f in RELEVANT_FREQS:
        highlight_freq(axes[idx][1], f, tol=0.05)

os.makedirs("./figures/", exist_ok=True)

axes[0][0].autoscale(True, axis="x", tight=True)
fig_1.savefig("./figures/target_temporal.svg")

axes[0][1].autoscale(True, axis="x", tight=True)
fig_2.savefig("./figures/target_frequency.svg")

### Independent Variable Visualization

- Not a lot changed in terms of periodicity
  - Slight monthly periodicity for a couple of variables, but this could be background noise or year-specific
- The second half of 05.2019 seems to be almost completely dark; this could either be a sensor malfunction or a prolonged storm
  - Checking weather records [online](https://www.timeanddate.com/weather/finland/lappeenranta/historic?month=5&year=2019), it seems the second explanation is more likely


In [ ]:
fig, axes = plt.subplots(
    len(VARIABLES), 2,
    sharex="col",
    figsize=(12, 6 * len(VARIABLES))
)
fig.tight_layout(pad=2)

for idx, var in enumerate(VARIABLES):
    sns.lineplot(
        DF,
        x="datetime",
        y=var,
        ax=axes[idx][0]
    )

    sns.lineplot(
        df_spectral,
        x="frequency",
        y=var + "_fft",
        ax=axes[idx][1]
    )
    axes[idx][1].set_xlabel("frequency [$h^{-1}$]")
    axes[idx][1].set_xscale("log")
    axes[idx][1].set_yscale("log")

    for f in RELEVANT_FREQS:
        highlight_freq(axes[idx][1], f, tol=0.1)

## Feature Engineering

### Wind

Direction logged in degrees. The discontinuity between 360 and 0 will confuse a model, so it's better to embed it along with wind speed and create 2 new columns: wind_cos and wind_sin (removed; mag+phase preserved)

In [ ]:
# DF = transfep_utils.transform_wind_features(DF)

### Datetime

As it's tradition, the most relative periodicities are encoded as circular variables. In this case: daily, weekly and yearly relative time values are encoded into pairs of columns using trigonometric functions.

In [ ]:
DF = transfep_utils.add_datetime_features(DF)

### Rebuild VARIABLES

In [ ]:
VARIABLES = [
    col for col in DF.columns
    if col != "datetime" and col not in TARGETS
]

## Feature Analysis

In [ ]:
x = StandardScaler().fit_transform(DF.select(VARIABLES).to_numpy())
y = StandardScaler().fit_transform(DF.select(TARGETS).to_numpy())

### PLS

- First Principal Component (most relevant for production) groups up radiation/sunshine variables, temperature, and annual season
- Second PC (most relevant for consumption) groups up all seasonalities and visibility
- Wind and Pressure are mostly irrelevant
- The Sine of DoY is not relevant, meaning that differentiating between Spring and Autumn is irrelevant to the model

In [ ]:
pls_regressor = PLSRegression(
    n_components=2,
    scale=False,
    max_iter=10000,
    tol=1e-6
).fit(x, y)

fig, axes = plt.subplots(
    2, 1,
    sharey=False,
    figsize=(12, len(TARGETS) * 6)
)

for idx in range(2):
    loadings_abs = np.abs(pls_regressor.x_loadings_[:, idx])
    axes[idx].barh(
        y=[var for _, var in sorted(zip(loadings_abs, VARIABLES))],
        width=sorted(loadings_abs),
    )
    axes[idx].set_title(f"PC{idx + 1}")

for idx, target in enumerate(TARGETS):
    print(
        f"PC loadings for {target}: {
            np.abs(
                pls_regressor.y_loadings_[
                    idx, :]
            )}"
    )
fig.tight_layout()

fig.savefig("./figures/pls_loadings.eps")

### Selection from GBR

By incorporating non-linearity, we observe the following:

- Sunshine is deemed redundant compared to radiation
- dow_cos is deemed irrelevant (no difference between Sunday and Thursday)
- On the other hand, dow_sin is heavily influential for consumption
- hour_sin redundant compared to hour_cos
- Consumption can be mostly modelled using time and temperature
- The ensemble benefits from accurate season information (yearly), as proven by doy_sin and doy_cos having about the same importance for the second model
- Production can be modelled using radiation (global and diffuse) and time information

In [ ]:
fig, axes = plt.subplots(
    len(TARGETS), 1,
    sharey=False,
    figsize=(12, len(TARGETS) * 6)
)

os.makedirs("./models/", exist_ok=True)

for idx, target in enumerate(TARGETS):
    if os.path.exists(f"./models/gbr_{target}.xz"):
        gbr = joblib.load(f"./models/gbr_{target}.xz")
    else:
        gbr = GradientBoostingRegressor(
            loss="squared_error",
            learning_rate=1e-3,
            n_estimators=1000,
            max_depth=None,
            subsample=1.,
            criterion="friedman_mse",
            init=PLSRegression(
                n_components=2,
                scale=False,
                max_iter=10000,
                tol=1e-6
            ).fit(x, y[:, idx]),
            validation_fraction=0.2,
            n_iter_no_change=10,
            tol=1e-6
        ).fit(x, y[:, idx])
        joblib.dump(
            value=gbr,
            filename=f"./models/gbr_{target}.xz",
            protocol=HIGHEST_PROTOCOL,
            compress=True
        )

    axes[idx].barh(
        y=[var for _, var in sorted(zip(gbr.feature_importances_, VARIABLES))],
        width=sorted(gbr.feature_importances_)
    )
    axes[idx].set_title(
        f"Feature Importance - {target}"
    )

fig.tight_layout()
fig.savefig("./figures/gbr_loadings.eps")